# Create Read Update Delete (CRUD) MySQL Database Dengan Python

Untuk mengakses MySQL, setidaknya kita bisa meminta bantuan dari driver [mysqlclient](https://mysqlclient.readthedocs.io). Lakukan [instalasi](https://github.com/PyMySQL/mysqlclient-python#install) dan import library tersebut. Untuk mempelajari tentang library tersebut dapat membuka [dokumentasinya](https://mysqlclient.readthedocs.io/user_guide.html#mysqldb) langsung.

In [1]:
import MySQLdb

Misal kita ingin membuat sebuah database untuk penyimpanan data anggota telegram di komunitas Surabaya.py. Jadi kita membutuhkan tabel anggota kurang lebih seperti ini:

|User ID|Nama|Email|Password|Telegram ID|Registered Date|Activation Key|
|--|--|--|--|--|--|--|
|int|varchar|varchar|varchar|varchar|date|varchar|
|Primary||Unique||Unique||Unique|

Sekarang kita coba terhubung dengan MySQL, membuat database dan membuat tabel anggota.

In [2]:
host = 'localhost'
mysql_username = 'root'
mysql_password = 'root'

database = 'surabayapy'
table_member = 'anggota'

In [3]:
db=MySQLdb.connect(host, mysql_username, mysql_password)
cursor = db.cursor()

In [4]:
# Membuat Database
sql = "CREATE DATABASE IF NOT EXISTS %s" % (database) 
cursor.execute(sql)

1

In [5]:
sql = 'USE `%s`' % (database)
cursor.execute(sql)

0

In [6]:
# Membuat Tabel
sql = '''CREATE TABLE IF NOT EXISTS `%s`.`%s` ( 
    `userid` INT NOT NULL AUTO_INCREMENT , 
    `name` VARCHAR(64) NOT NULL , 
    `email` VARCHAR(64) NOT NULL , 
    `password` VARCHAR(64) NOT NULL , 
    `telegram_id` VARCHAR(64) ,
    `registered_date` DATE NOT NULL , 
    `key` VARCHAR(64) , 
    PRIMARY KEY (`userid`),
    UNIQUE (`email`),
    UNIQUE (`telegram_id`),
    UNIQUE (`key`)
    ) ENGINE = InnoDB;
   ''' % (database, table_member)
cursor.execute(sql)

0

In [7]:
from datetime import datetime
import hashlib

In [8]:
# Read Account If Already Registered

def check_not_registered(telegram_id, email):
    sql = '''
                select `name` from `%s` where `telegram_id`='%s' or `email`='%s';
            ''' % (table_member, telegram_id, email)
    response = cursor.execute(sql)
    if (response > 0):
        response = cursor.fetchone()
        print("Hello %s, check your email for activating your account" % response[0])
        return False
    
    return True

In [9]:
# Supporing Function

def sha256(password):
    password = password.encode('UTF-8')
    hash_object = hashlib.sha256(password)
    hex_dig = hash_object.hexdigest()
    return hex_dig

def md5(text):
    from random import randint
    salt = str(randint(1, 7829))
    text = salt+text
    text = text.encode('UTF-8')
    hash_object = hashlib.md5(text)
    hex_dig = hash_object.hexdigest()
    return hex_dig

In [10]:
# Create Account
def register(name, email, password, telegram_id):
    if check_not_registered(telegram_id, email):
        password = sha256(password)
        key = md5(email + str(datetime.utcnow()))
        # userid, name, email, password, telegram_id, registered_date, activation_key
        sql = '''
                    insert into `%s` values (NULL, '%s', '%s', '%s', '%s', CURRENT_DATE(), '%s');
                ''' % (table_member, name, email, password, telegram_id, key) 
        cursor.execute(sql)
        print('Account activation code: "%s" for @%s' % (key, telegram_id))
    
        db.commit()

In [11]:
# Update Account
def activation(key, telegram_id):
    sql = '''
              update `%s` set `key` = NULL where `telegram_id` = '%s' and `key` = '%s';
          ''' % (table_member, telegram_id, key)
    response = cursor.execute(sql)
    db.commit()
    if (response > 0):
        print('Activation Success')
    else:
        print('Activation failed. Please try again.')

In [12]:
# Delete Account
def request_delete(email, password, telegram_id):
    password = sha256(password)
    key = md5(email + str(datetime.utcnow()))
    sql = '''
          update `%s` set `key` = '%s' where `email` = '%s' and `telegram_id` = '%s';
      ''' % (table_member, key, email, telegram_id)
    response = cursor.execute(sql)
    db.commit()
    if (response > 0):
        print('Confirm your deletion with key: %s' % (key))
    else:
        print('Deletion request rejected')
    
def confirm_delete(email, telegram_id, key):
    sql = '''
              delete from `%s` where `email` = '%s' and `telegram_id` = '%s' and `key` = '%s';
          ''' % (table_member, email, telegram_id, key)
    
    response = cursor.execute(sql)
    db.commit()
    if (response > 0):
        print('%s has been deleted' % (telegram_id))
    else:
        print('Deletion rejected')


In [13]:
name = 'Tegar Imansyah'
email = 'tegar@imansyah.name'
password = 'cobapassword'
telegram_id = 'tegarimansyah'

register(name, email, password, telegram_id)

Account activation code: "04e4b6a33ef4a7af1d33225e9f5ad270" for @tegarimansyah


In [14]:
activation_code = '04e4b6a33ef4a7af1d33225e9f5ad270'
activation(activation_code, telegram_id)

Activation Success


In [15]:
request_delete(email,password,telegram_id)

Confirm your deletion with key: 1034500e6126084ac7202fce5f877871


In [16]:
deletion_key = '1034500e6126084ac7202fce5f877871'
confirm_delete(email, telegram_id, deletion_key)

tegarimansyah has been deleted


In [ ]:
# Just in case you need to restart

# sql = "DROP DATABASE IF EXISTS `%s`;" % (database)
# cursor.execute(sql)